In [1]:
# Causual inference of price elasticity of demand is the process of identifying cause-effect relationships between variables.
# How to interpret value of price elasticity of demand ?
    # |PED| <1 - inelastic (demand LESS sensitive to price changes)
    # |PED| >1 - elastic (demand MORE sensitive to price changes)
    # +ve - demand increase with price increase
    # -ve - demand decrease with price increase (typical case)
# Usually, the sign of elasticity value is ignored for easier interpretation. We assume inverse relationship of demand and price
# Treatment variable: price
# Outcome variable: demand

In [2]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [3]:
import xgboost
print(xgboost.__version__)

2.1.3


In [4]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

### Derive log transformed PRICE for PED estimation

In [6]:
# Read data
data = pd.read_excel(r"C:\Users\Dell\Downloads\FINAL SUBSET_preprocessed_data.xlsx")  
df1 = data.copy()


In [7]:
# Derive log transformed PRICE
df1['PRICE_LOG'] = np.log(df1['PRICE'])
df1[['PRICE', 'PRICE_LOG']]

,PRICE,PRICE_LOG
0,4.34,1.467874
1,3.31,1.196948
2,3.91,1.363537
3,4.16,1.425515
4,5.73,1.745716
...,...,...
662442,2.50,0.916291
662443,4.41,1.483875
662444,1.94,0.662688
662445,4.12,1.415853


In [8]:
temp = df1[(df1['STORE_ID'] == 389) & (df1['UPC'] == 1111038078)]
temp[['WEEK_END_DATE', 'YEAR', 'WEEK_NUM', 'STORE_ID', 'UPC', 'UNIT_SALES', 'PRICE']]

,WEEK_END_DATE,YEAR,WEEK_NUM,STORE_ID,UPC,UNIT_SALES,PRICE
1173,2009-01-14,2009,3,389,1111038078,6,2.59
4586,2009-01-19,2009,4,389,1111038078,412,4.02
4828,2009-01-21,2009,4,389,1111038078,28,0.98
8568,2009-01-28,2009,5,389,1111038078,24,1.00
11699,2009-01-29,2009,5,389,1111038078,59,3.46
...,...,...,...,...,...,...,...
646473,2011-12-14,2011,50,389,1111038078,6,1.69
650483,2011-12-21,2011,51,389,1111038078,4,1.69
650484,2011-12-21,2011,51,389,1111038078,216,2.21
654355,2011-12-28,2011,52,389,1111038078,5,1.69


### Double Machine Learning Model Building

In [10]:
# Define confounding variables
confounders =['YEAR','WEEK_NUM', 'FEATURE', 'DISPLAY', 'SEG_VALUE_NAME_ORDINAL', 'SALES_AREA_SIZE_NUM'] 

treatment_r2_scores = []
treatment_mae_scores = []
outcome_r2_scores = []
outcome_mae_scores = []

def estimate_price_elasticity(store_id, upc):
    global treatment_r2_scores, treatment_mae_scores, outcome_r2_scores, outcome_mae_scores

    # Extract subset data for each store-upc combination
    subset_df = df1[(df1['STORE_ID'] == store_id) & (df1['UPC'] == upc)]

    # Define treatment (PRICE) and outcome variable (UNIT SALES)
    X_train_treatment = subset_df[confounders] 
    X_train_outcome = subset_df[['PRICE_LOG'] + confounders] 

    Y_treatment = subset_df['PRICE_LOG']
    Y_outcome = subset_df['UNIT_SALES_LOG']

    # First stage estimation: Regress treatment (PRICE_LOG) and outcome (UNIT_SALES_LOG) on cofounders
    treatment_model = XGBRegressor(random_state=42)   
    outcome_model = XGBRegressor(random_state=42)
    
    treatment_model.fit(X_train_treatment, Y_treatment)
    outcome_model.fit(X_train_outcome, Y_outcome)
 
    Y_treatment_pred = treatment_model.predict(X_train_treatment)
    Y_outcome_pred = outcome_model.predict(X_train_outcome)

    # Derive residuals to approximate percentage changes of PRICE and UNIT_SALES. 
    log_price_residuals = Y_treatment - Y_treatment_pred

    log_demand_residuals = Y_outcome - Y_outcome_pred

    # print("R² (Treatment model): ", r2_score(np.exp(Y_treatment), np.exp(Y_treatment_pred)))
    # print("MAE (Treatment model): ", mean_absolute_error(np.exp(Y_treatment), np.exp(Y_treatment_pred)))
    # print("R² (Ouctome model): ", r2_score(np.exp(Y_outcome), np.exp(Y_outcome_pred)))
    # print("MAE (Ouctome model): ", mean_absolute_error(np.exp(Y_outcome), np.exp(Y_outcome_pred)))
    
    treatment_r2 = r2_score(np.exp(Y_treatment), np.exp(Y_treatment_pred))
    treatment_mae = mean_absolute_error(np.exp(Y_treatment), np.exp(Y_treatment_pred))
    treatment_r2_scores.append(treatment_r2)
    treatment_mae_scores.append(treatment_mae)

    outcome_r2 = r2_score(np.exp(Y_outcome), np.exp(Y_outcome_pred))
    outcome_mae = mean_absolute_error(np.exp(Y_outcome), np.exp(Y_outcome_pred))
    outcome_r2_scores.append(outcome_r2)
    outcome_mae_scores.append(outcome_mae)

    # Second stage estimation [log-log regression model]: Regress treatment (X) and outcome residulas (Y)  
    log_linear_model = LinearRegression()
    log_linear_model.fit(log_price_residuals.values.reshape(-1, 1), log_demand_residuals) # reshape input X to 2D array
    price_elasticity = log_linear_model.coef_[0]

    return price_elasticity 

### Price Elasticify of Demand Estimation (Loop each store-product pair)

In [12]:
df1.head()

,WEEK_END_DATE,YEAR,WEEK_NUM,STORE_ID,UPC,UNIT_SALES,UNIT_SALES_LOG,PRICE,PRICE_SQRT,BASE_PRICE,...,FEATURE,DISPLAY,PROMOTION_SUPPORT,CATEGORY,SUB_CATEGORY,STORE_NAME,SEG_VALUE_NAME,SEG_VALUE_NAME_ORDINAL,SALES_AREA_SIZE_NUM,PRICE_LOG
0,2008-12-15,2008,51,387,88491201426,284,5.648974,4.34,2.083267,4.97,...,0,0,UNSUPPORTED,COLD CEREAL,ADULT CEREAL,TOWN & COUNTRY,UPSCALE,2,44547,1.467874
1,2008-12-15,2008,51,2523,2840002333,379,5.937536,3.31,1.819341,3.92,...,0,1,UNSUPPORTED,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373,1.196948
2,2008-12-15,2008,51,4245,1600027564,125,4.828314,3.91,1.977372,4.85,...,1,0,SUPPORTED,COLD CEREAL,ALL FAMILY CEREAL,LAWRENCEBURG,MAINSTREAM,1,58563,1.363537
3,2008-12-15,2008,51,4489,3700031613,55,4.007333,4.16,2.039608,5.76,...,1,0,SUPPORTED,ORAL HYGIENE PRODUCTS,MOUTHWASH/RINSES AND SPRAYS,SWEETWATER PLAZA,UPSCALE,2,49030,1.425515
4,2008-12-15,2008,51,4489,4116709448,484,6.182085,5.73,2.393742,7.12,...,0,0,UNSUPPORTED,ORAL HYGIENE PRODUCTS,MOUTHWASH/RINSES AND SPRAYS,SWEETWATER PLAZA,UPSCALE,2,49030,1.745716


In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662447 entries, 0 to 662446
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   WEEK_END_DATE           662447 non-null  datetime64[ns]
 1   YEAR                    662447 non-null  int64         
 2   WEEK_NUM                662447 non-null  int64         
 3   STORE_ID                662447 non-null  int64         
 4   UPC                     662447 non-null  int64         
 5   UNIT_SALES              662447 non-null  int64         
 6   UNIT_SALES_LOG          662447 non-null  float64       
 7   PRICE                   662447 non-null  float64       
 8   PRICE_SQRT              662447 non-null  float64       
 9   BASE_PRICE              662447 non-null  float64       
 10  BASE_PRICE_SQRT         661894 non-null  float64       
 11  DISCOUNT_RATE           662447 non-null  float64       
 12  FEATURE                 662447

In [14]:
df1['PRICE_SQRT']=df1['PRICE_SQRT'].fillna(df1['PRICE_SQRT'].median())

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662447 entries, 0 to 662446
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   WEEK_END_DATE           662447 non-null  datetime64[ns]
 1   YEAR                    662447 non-null  int64         
 2   WEEK_NUM                662447 non-null  int64         
 3   STORE_ID                662447 non-null  int64         
 4   UPC                     662447 non-null  int64         
 5   UNIT_SALES              662447 non-null  int64         
 6   UNIT_SALES_LOG          662447 non-null  float64       
 7   PRICE                   662447 non-null  float64       
 8   PRICE_SQRT              662447 non-null  float64       
 9   BASE_PRICE              662447 non-null  float64       
 10  BASE_PRICE_SQRT         661894 non-null  float64       
 11  DISCOUNT_RATE           662447 non-null  float64       
 12  FEATURE                 662447

In [16]:
df1['BASE_PRICE_SQRT']=df1['BASE_PRICE_SQRT'].fillna(df1['BASE_PRICE_SQRT'].median())

In [17]:
df1['PRICE_LOG']=df1['PRICE_LOG'].fillna(df1['PRICE_LOG'].median())

In [18]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662447 entries, 0 to 662446
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   WEEK_END_DATE           662447 non-null  datetime64[ns]
 1   YEAR                    662447 non-null  int64         
 2   WEEK_NUM                662447 non-null  int64         
 3   STORE_ID                662447 non-null  int64         
 4   UPC                     662447 non-null  int64         
 5   UNIT_SALES              662447 non-null  int64         
 6   UNIT_SALES_LOG          662447 non-null  float64       
 7   PRICE                   662447 non-null  float64       
 8   PRICE_SQRT              662447 non-null  float64       
 9   BASE_PRICE              662447 non-null  float64       
 10  BASE_PRICE_SQRT         662447 non-null  float64       
 11  DISCOUNT_RATE           662447 non-null  float64       
 12  FEATURE                 662447

In [19]:
# Extract unique store-upc combinations 
unique_store_upc = df1[['STORE_ID', 'UPC']].drop_duplicates()
unique_store_upc['PRICE_ELASTICITY'] = np.nan
print(len(unique_store_upc), "unique store-upc combinations\n")

for index, row in unique_store_upc.iterrows():
    store_id = int(row['STORE_ID'])
    upc = int(row['UPC'])
    price_elasticity = estimate_price_elasticity(store_id, upc)
    print(price_elasticity)
    unique_store_upc.at[index, 'PRICE_ELASTICITY'] = price_elasticity

# For interpretability, use the absolute value. Ignore the sign by assuming all products follow Law of Demand (price increase, demand decrease)
unique_store_upc['ABS_PRICE_ELASTICITY'] = unique_store_upc['PRICE_ELASTICITY'].abs()   
unique_store_upc = unique_store_upc.sort_values(by=['UPC', 'STORE_ID'])

#unique_store_upc.to_excel('Dataset/Price Elasticity.xlsx', index=False)

4235 unique store-upc combinations

0.006672873234973592
-0.00036135838409478066
-0.0012692481575151925
0.026341454503414234
0.01900469683708297
-0.019956034954018095
0.012647769295027901
0.049997622304716605
0.008191530279570437
-0.016407233718950646
0.00030805276738134235
-0.005603869651472904
0.03463813003965161
0.023073581475531475
0.011292860537548112
0.03177804861183497
-0.003003728800302038
0.00022854363848812175
-0.022135427428877672
-0.001234116793940786
-0.005140778602801281
-0.003690726577817099
0.0008027029344334869
-0.002504093218202708
0.01129468082800715
-0.0033397275005570748
0.0007903533551401943
-0.0014705711398811413
-0.002134616209296506
0.0018728236572014643
-0.011800257759605522
-0.015111477013740007
-0.06082256907766627
0.0012680687964104701
0.0006745048497935007
0.007572341843512816
0.004917710341116068
0.0029772360626355984
-0.00910929746065526
-0.0029919965868061144
0.0013684139370255032
0.0010574583872146066
0.008112888101086704
-0.021419246213533225
-0.02813

In [20]:
def get_average_metrics():
    avg_treatment_r2 = np.mean(treatment_r2_scores)
    avg_treatment_mae = np.mean(treatment_mae_scores)
    avg_outcome_r2 = np.mean(outcome_r2_scores)
    avg_outcome_mae = np.mean(outcome_mae_scores)
    return avg_treatment_r2, avg_treatment_mae, avg_outcome_r2, avg_outcome_mae  

In [21]:
avg_treatment_r2, avg_treatment_mae, avg_outcome_r2, avg_outcome_mae = get_average_metrics()
print("Treatment Model")
print("Average R²: ",avg_treatment_r2)
print("Average MAE: ", avg_treatment_mae)

print("\nOutcome Model")
print("Average R²: ", avg_outcome_r2)
print("Average MAE: ", avg_outcome_mae)

Treatment Model
Average R²:  0.8846027371083778
Average MAE:  0.08621864387366636

Outcome Model
Average R²:  0.9995851516885192
Average MAE:  0.3221896813316651


In [22]:
elasticity = pd.read_excel(r'C:\Users\Dell\Downloads\Price_Elasticity.xlsx')
product = pd.read_excel(r'C:\Users\Dell\Downloads\dunnhumby - Breakfast at the Frat.xlsx',sheet_name='dh_Products_Lookup')
store = pd.read_excel(r'C:\Users\Dell\Downloads\dunnhumby - Breakfast at the Frat.xlsx',sheet_name='dh_Store_Lookup')
elasticity

,STORE_ID,UPC,PRICE_ELASTICITY,ABS_PRICE_ELASTICITY
0,367,1111009477,0.063871,0.063871
1,387,1111009477,0.058811,0.058811
2,389,1111009477,0.009605,0.009605
3,613,1111009477,0.071135,0.071135
4,623,1111009477,-0.028330,0.028330
...,...,...,...,...
2840,26981,88491212971,0.082167,0.082167
2841,26983,88491212971,0.050827,0.050827
2842,27175,88491212971,-0.047577,0.047577
2843,28909,88491212971,0.164867,0.164867


In [23]:
print("Total Store-Product Pairs: ", len(elasticity), '\n')  # 2845
elasticity[elasticity['PRICE_ELASTICITY'] < 0]  # 1385 (follow law of demand)
elasticity[elasticity['PRICE_ELASTICITY'] > 0]  # 1459 (contrast law of demand)

perfectly_inelastic = elasticity[elasticity['ABS_PRICE_ELASTICITY'] == 0] #(perfectly inelastic: price changes have no impact on demand)
print("Perfectly Inelastic Demand (=0): ", len(perfectly_inelastic)) 

inelastic_store_product = elasticity[(elasticity['ABS_PRICE_ELASTICITY'] > 0) & (elasticity['ABS_PRICE_ELASTICITY'] < 1)]
print("Inelastic Demand (<1): ", len(inelastic_store_product))   

highly_inelastic_store_product = elasticity[(elasticity['ABS_PRICE_ELASTICITY'] > 0) & (elasticity['ABS_PRICE_ELASTICITY'] <= 0.1)]
print("Highly Inelastic Demand (>0 and <=0.1): ", len(highly_inelastic_store_product))   

moderately_inelastic_store_product = elasticity[(elasticity['ABS_PRICE_ELASTICITY'] > 0.1) & (elasticity['ABS_PRICE_ELASTICITY'] <= 0.5)]
print("Moderately Inelastic Demand (>0.1 and <=0.5): ", len(moderately_inelastic_store_product))  

slightly_inelastic_store_product = elasticity[(elasticity['ABS_PRICE_ELASTICITY'] > 0.5) & (elasticity['ABS_PRICE_ELASTICITY'] < 1)]
print("Slightly Inelastic Demand (>0.5 and <1): ", len(slightly_inelastic_store_product))  

elastic_store_product = elasticity[elasticity['ABS_PRICE_ELASTICITY'] > 1]   
print("Elastic Demand (>1): ", len(elastic_store_product))   # 44 store-product

Total Store-Product Pairs:  2845 

Perfectly Inelastic Demand (=0):  1
Inelastic Demand (<1):  2800
Highly Inelastic Demand (>0 and <=0.1):  1341
Moderately Inelastic Demand (>0.1 and <=0.5):  1272
Slightly Inelastic Demand (>0.5 and <1):  187
Elastic Demand (>1):  44


In [24]:
# perfectly_inelastic
data[(data['STORE_ID'] == 2523) & (data['UPC'] == 7797502248)]


,WEEK_END_DATE,YEAR,WEEK_NUM,STORE_ID,UPC,UNIT_SALES,UNIT_SALES_LOG,PRICE,PRICE_SQRT,BASE_PRICE,...,DISCOUNT_RATE,FEATURE,DISPLAY,PROMOTION_SUPPORT,CATEGORY,SUB_CATEGORY,STORE_NAME,SEG_VALUE_NAME,SEG_VALUE_NAME_ORDINAL,SALES_AREA_SIZE_NUM
15731,2009-02-07,2009,6,2523,7797502248,155,5.043425,4.67,2.161018,5.54,...,0.157040,1,0,UNSUPPORTED,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
51473,2009-04-15,2009,16,2523,7797502248,387,5.958425,5.24,2.289105,6.13,...,0.145188,1,1,UNSUPPORTED,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
78417,2009-05-31,2009,22,2523,7797502248,496,6.206576,2.22,1.489966,3.44,...,0.354651,0,0,UNSUPPORTED,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
86767,2009-06-16,2009,25,2523,7797502248,316,5.755742,2.77,1.664332,4.29,...,0.354312,0,1,UNSUPPORTED,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
169980,2009-11-04,2009,45,2523,7797502248,445,6.098074,4.18,2.044505,5.99,...,0.302170,0,0,UNSUPPORTED,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
172925,2009-11-05,2009,45,2523,7797502248,106,4.663439,2.49,1.577973,2.86,...,0.129371,0,0,UNSUPPORTED,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
177371,2009-11-14,2009,46,2523,7797502248,336,5.817111,2.58,1.606238,3.15,...,0.180952,0,1,SUPPORTED,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
177484,2009-11-15,2009,46,2523,7797502248,323,5.777652,2.33,1.526434,3.83,...,0.391645,1,0,SUPPORTED,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
178402,2009-11-18,2009,47,2523,7797502248,97,4.574711,2.63,1.621727,4.11,...,0.360097,0,0,NONE,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
215627,2010-01-16,2010,2,2523,7797502248,329,5.796058,3.58,1.892089,5.25,...,0.318095,1,1,UNSUPPORTED,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373


In [25]:
def get_store_product(store_id, upc):
    store_row = store[store['STORE_ID'] == store_id]
    store_info = {
        'STORE_ID': int(store_id),
        'STORE_NAME': store_row['STORE_NAME'].iloc[0],
        'STORE_SEGMENT_VALUE': store_row['SEG_VALUE_NAME'].iloc[0]
    }
    upc_row = product[product['UPC'] == upc]
    product_info = {
        'UPC': int(upc),
        'PRODUCT_CATEGORY': upc_row['CATEGORY'].iloc[0],
        'PRODUCT_SUB_CATEGORY': upc_row['SUB_CATEGORY'].iloc[0],
        'PRODUCT_MANUFACTURER': upc_row['MANUFACTURER'].iloc[0]
    }
    store_product_info = {**store_info, **product_info}
    return store_product_info

In [26]:
highly_inelastic_store_product_ls = []
for index, row in highly_inelastic_store_product.reset_index().iterrows():
    store_id = row['STORE_ID']
    upc = row['UPC']
    store_product_info = get_store_product(store_id, upc)
    highly_inelastic_store_product_ls.append(store_product_info)
highly_inelastic_store_product_df = pd.DataFrame(highly_inelastic_store_product_ls)


moderately_inelastic_store_product_ls = []
for index, row in moderately_inelastic_store_product.reset_index().iterrows():
    store_id = row['STORE_ID']
    upc = row['UPC']
    store_product_info = get_store_product(store_id, upc)
    moderately_inelastic_store_product_ls.append(store_product_info)
moderately_inelastic_store_product_df = pd.DataFrame(moderately_inelastic_store_product_ls)


slightly_inelastic_store_product_ls = []
for index, row in slightly_inelastic_store_product.reset_index().iterrows():
    store_id = row['STORE_ID']
    upc = row['UPC']
    store_product_info = get_store_product(store_id, upc)
    slightly_inelastic_store_product_ls.append(store_product_info)
slightly_inelastic_store_product_df = pd.DataFrame(slightly_inelastic_store_product_ls)


elastic_store_product_ls = []
for index, row in elastic_store_product.reset_index().iterrows():
    store_id = row['STORE_ID']
    upc = row['UPC']
    store_product_info = get_store_product(store_id, upc)
    elastic_store_product_ls.append(store_product_info)
elastic_store_product_df = pd.DataFrame(elastic_store_product_ls)

In [27]:
print(highly_inelastic_store_product_df['PRODUCT_CATEGORY'].value_counts())
print()
print(highly_inelastic_store_product_df['PRODUCT_SUB_CATEGORY'].value_counts())

PRODUCT_CATEGORY
COLD CEREAL              620
BAG SNACKS               285
FROZEN PIZZA             264
ORAL HYGIENE PRODUCTS    172
Name: count, dtype: int64

PRODUCT_SUB_CATEGORY
ALL FAMILY CEREAL              331
PRETZELS                       285
PIZZA/PREMIUM                  264
KIDS CEREAL                    172
MOUTHWASHES (ANTISEPTIC)       152
ADULT CEREAL                   117
MOUTHWASH/RINSES AND SPRAYS     20
Name: count, dtype: int64


In [28]:
highly_inelastic_store_product_df[(highly_inelastic_store_product_df['PRODUCT_CATEGORY'] == 'FROZEN PIZZA') 
                                  & (highly_inelastic_store_product_df['STORE_ID'] == 387)]

,STORE_ID,STORE_NAME,STORE_SEGMENT_VALUE,UPC,PRODUCT_CATEGORY,PRODUCT_SUB_CATEGORY,PRODUCT_MANUFACTURER
407,387,TOWN & COUNTRY,UPSCALE,1111087395,FROZEN PIZZA,PIZZA/PREMIUM,PRIVATE LABEL
469,387,TOWN & COUNTRY,UPSCALE,1111087396,FROZEN PIZZA,PIZZA/PREMIUM,PRIVATE LABEL
1021,387,TOWN & COUNTRY,UPSCALE,7192100337,FROZEN PIZZA,PIZZA/PREMIUM,TOMBSTONE
1041,387,TOWN & COUNTRY,UPSCALE,7192100339,FROZEN PIZZA,PIZZA/PREMIUM,TOMBSTONE


In [29]:
print(moderately_inelastic_store_product_df['PRODUCT_CATEGORY'].value_counts())
print()
print(moderately_inelastic_store_product_df['PRODUCT_SUB_CATEGORY'].value_counts())

PRODUCT_CATEGORY
COLD CEREAL              488
ORAL HYGIENE PRODUCTS    296
BAG SNACKS               250
FROZEN PIZZA             238
Name: count, dtype: int64

PRODUCT_SUB_CATEGORY
PRETZELS                       250
MOUTHWASHES (ANTISEPTIC)       250
PIZZA/PREMIUM                  238
KIDS CEREAL                    200
ALL FAMILY CEREAL              187
ADULT CEREAL                   101
MOUTHWASH/RINSES AND SPRAYS     46
Name: count, dtype: int64


In [30]:
moderately_inelastic_store_product_df[
    (moderately_inelastic_store_product_df['PRODUCT_CATEGORY'] == 'COLD CEREAL') 
    & (moderately_inelastic_store_product_df['STORE_ID'] == 387)]

,STORE_ID,STORE_NAME,STORE_SEGMENT_VALUE,UPC,PRODUCT_CATEGORY,PRODUCT_SUB_CATEGORY,PRODUCT_MANUFACTURER
141,387,TOWN & COUNTRY,UPSCALE,1111085319,COLD CEREAL,ALL FAMILY CEREAL,PRIVATE LABEL
160,387,TOWN & COUNTRY,UPSCALE,1111085345,COLD CEREAL,ADULT CEREAL,PRIVATE LABEL
257,387,TOWN & COUNTRY,UPSCALE,1600027528,COLD CEREAL,ALL FAMILY CEREAL,GENERAL MI
302,387,TOWN & COUNTRY,UPSCALE,1600027564,COLD CEREAL,ALL FAMILY CEREAL,GENERAL MI
466,387,TOWN & COUNTRY,UPSCALE,3000006610,COLD CEREAL,KIDS CEREAL,QUAKER
650,387,TOWN & COUNTRY,UPSCALE,3800031838,COLD CEREAL,KIDS CEREAL,KELLOGG
666,387,TOWN & COUNTRY,UPSCALE,3800039118,COLD CEREAL,KIDS CEREAL,KELLOGG
1196,387,TOWN & COUNTRY,UPSCALE,88491201427,COLD CEREAL,ADULT CEREAL,POST FOODS


In [31]:
print(slightly_inelastic_store_product_df['PRODUCT_CATEGORY'].value_counts())
print()
print(slightly_inelastic_store_product_df['PRODUCT_SUB_CATEGORY'].value_counts())

PRODUCT_CATEGORY
ORAL HYGIENE PRODUCTS    62
BAG SNACKS               51
COLD CEREAL              39
FROZEN PIZZA             35
Name: count, dtype: int64

PRODUCT_SUB_CATEGORY
MOUTHWASHES (ANTISEPTIC)       53
PRETZELS                       51
PIZZA/PREMIUM                  35
ALL FAMILY CEREAL              17
ADULT CEREAL                   11
KIDS CEREAL                    11
MOUTHWASH/RINSES AND SPRAYS     9
Name: count, dtype: int64


In [32]:
slightly_inelastic_store_product_df[
slightly_inelastic_store_product_df['PRODUCT_CATEGORY'] == 'ORAL HYGIENE PRODUCTS']

,STORE_ID,STORE_NAME,STORE_SEGMENT_VALUE,UPC,PRODUCT_CATEGORY,PRODUCT_SUB_CATEGORY,PRODUCT_MANUFACTURER
0,6187,ST. MARYS,MAINSTREAM,1111038078,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),PRIVATE LABEL
1,13609,VANDALIA,MAINSTREAM,1111038080,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),PRIVATE LABEL
2,15547,WOODLAWN,MAINSTREAM,1111038080,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),PRIVATE LABEL
3,23067,WHIPP & BIGGER,MAINSTREAM,1111038080,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),PRIVATE LABEL
4,26981,SPRINGFIELD,MAINSTREAM,1111038080,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),PRIVATE LABEL
...,...,...,...,...,...,...,...
169,23061,LIBERTY TWP.,MAINSTREAM,31254742835,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),WARNER
170,25001,PINEWOOD,VALUE,31254742835,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),WARNER
171,25233,ANTOINE TOWN CENTER,VALUE,31254742835,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),WARNER
172,25261,CROWLEY,MAINSTREAM,31254742835,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),WARNER


In [33]:
print(elastic_store_product_df['PRODUCT_CATEGORY'].value_counts())
print()
print(elastic_store_product_df['PRODUCT_SUB_CATEGORY'].value_counts())

PRODUCT_CATEGORY
BAG SNACKS               29
COLD CEREAL               8
ORAL HYGIENE PRODUCTS     5
FROZEN PIZZA              2
Name: count, dtype: int64

PRODUCT_SUB_CATEGORY
PRETZELS                       29
ALL FAMILY CEREAL               4
MOUTHWASHES (ANTISEPTIC)        3
MOUTHWASH/RINSES AND SPRAYS     2
KIDS CEREAL                     2
PIZZA/PREMIUM                   2
ADULT CEREAL                    2
Name: count, dtype: int64


In [34]:
elastic_store_product_df[(elastic_store_product_df['PRODUCT_CATEGORY'] == 'BAG SNACKS') ]

,STORE_ID,STORE_NAME,STORE_SEGMENT_VALUE,UPC,PRODUCT_CATEGORY,PRODUCT_SUB_CATEGORY,PRODUCT_MANUFACTURER
4,4521,PARKWAY SQUARE S/C,MAINSTREAM,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
5,6187,ST. MARYS,MAINSTREAM,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
6,6431,AT WARD ROAD,VALUE,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
7,11967,NORTHBOROUGH SQ,VALUE,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
8,12011,SHERMAN,MAINSTREAM,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
9,13859,MCKINNEY,MAINSTREAM,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
10,15755,KROGER JUNCTION S/C,VALUE,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
11,17599,KEARNEY,VALUE,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
12,17615,SUGARLAND,MAINSTREAM,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
13,19523,DUNCANVILLE,VALUE,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
